In [44]:
import boto3
import pprint
import traceback

pp = pprint.PrettyPrinter(indent=2)

client = boto3.client('bedrock-agent-runtime', region_name='us-west-2')

sessionId = None

def query_bedrock_agent(kbId, input_text, model_arn, session_id=None):
    global sessionId

    params = {
        'input': {
            'text': input_text
        },
        'retrieveAndGenerateConfiguration': {
            'knowledgeBaseConfiguration': {
                'generationConfiguration': {
                    'guardrailConfiguration': {
                        'guardrailId': '6nsnq854bfxa',
                        'guardrailVersion': 'DRAFT'
                    },
                'promptTemplate': {
                    'textPromptTemplate': 'You are a question answering agent. I will provide you with a set of search results. The user will provide you with a question. Your job is to answer the user question using only information from the search results.'
                 }
                },
                'knowledgeBaseId': kbId,
                'modelArn': model_arn,
                 'orchestrationConfiguration': {
                     'queryTransformationConfiguration': {
                         'type': 'QUERY_DECOMPOSITION'
                     },
                'promptTemplate': {
                    'textPromptTemplate': 'You are a query creation agent. You will be provided with a function and a description of what it searches over. The user will provide you a question, and your job is to determine the optimal query to use based on the user question.'
                }
                  },
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'numberOfResults': 5,
                        'overrideSearchType': 'HYBRID',
                        'rerankingConfiguration': {
                        'bedrockRerankingConfiguration': {
                            'metadataConfiguration': {
                                'selectionMode': 'ALL',
                            },
                            'modelConfiguration': {
                                'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.rerank-v1:0'
                            },
                            'numberOfRerankedResults': 1
                        },
                        'type': 'BEDROCK_RERANKING_MODEL'
                      }
                    }
                }
            },
            'type': 'KNOWLEDGE_BASE'
        },
        'sessionConfiguration':{
        'kmsKeyArn': 'arn:aws:kms:us-west-2:664418992117:key/6e3b3e5f-9fab-40ee-aa0d-2b7bd35a1621'
        }
    }

    if session_id:
        params['sessionId'] = session_id

    try:
        response = client.retrieve_and_generate_stream(**params)
#        print(f"SessionId: {response['sessionId']}")
        for event in response['stream']:
            # Each event is a dictionary containing the event data
            output_text = event.get('output', 'text')
            print(f"Output_text: {output_text}")
            print(f"stream_citation: {stream_citation}")

    except Exception as e:
        traceback.print_exc()
        return

    # Process and display the response
    try:
        if 'output' in response and 'text' in response['output']:
            generated_text = response['output']['text']
            pp.pprint(generated_text)
        else:
            print("No generated text found in response")

        if 'guardrailAction' in response:
            guardRailAction = response['guardrailAction']
            pp.pprint(guardRailAction)
        else:
            print("No guardrail action found in response")

        if 'sessionId' in response:
            sessionId = response['sessionId']
            pp.pprint(sessionId)
        else:
            print("No session ID found in response")

        if 'citations' in response:
            citations = response["citations"]
            contexts = []
            for citation in citations:
                if 'retrievedReferences' in citation:
                    retrievedReferences = citation["retrievedReferences"]
                    for reference in retrievedReferences:
                        if 'content' in reference and 'text' in reference['content']:
                            contexts.append(reference["content"]["text"])
            if contexts:
                pp.pprint(contexts)
            else:
                print("No contexts found in response")
        else:
            print("No citations found in response")
    except Exception as e:
        print(f"Error: {str(e)}")

# Example usage
if __name__ == "__main__":
    input_text = "what is food safety?"
    kbId='QJRLZNDR9U'
#     model_arn = 'arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-5-sonnet-20241022-v2:0'
    model_arn = 'arn:aws:bedrock:us-west-2:664418992117:imported-model/urypmrpq5gjs'
    query_bedrock_agent(kbId, input_text, model_arn)


# In[ ]:






Traceback (most recent call last):
  File "/tmp/ipykernel_3492/1154278845.py", line 69, in query_bedrock_agent
    response = client.retrieve_and_generate_stream(**params)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py", line 569, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py", line 1023, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.ValidationException: An error occurred (ValidationException) when calling the RetrieveAndGenerateStream operation: [textPromptTemplate::generationConfiguration] must contain the $search_results$ placeholder variable.


In [46]:
import boto3
import pprint
import traceback

pp = pprint.PrettyPrinter(indent=2)

client = boto3.client('bedrock-agent-runtime', region_name='us-west-2')

sessionId = None

def query_bedrock_agent(kbId, input_text, model_arn, session_id=None):
    global sessionId

    # Define request parameters
    params = {
        'input': {
            'text': input_text
        },
        'retrieveAndGenerateConfiguration': {
            'knowledgeBaseConfiguration': {
                'generationConfiguration': {
                    'guardrailConfiguration': {
                        'guardrailId': '6nsnq854bfxa',
                        'guardrailVersion': 'DRAFT'
                    },
                    'promptTemplate': {
                        'textPromptTemplate': '''You are a question answering agent. I will provide you with a set of search results. The user will provide you with a question. Your job is to answer the user question using only information from the search results.
                        
                        Search Results:
                        $search_results$
                        
                        User Question: {user_question}
                        
                        Answer:'''
                    }
                },
                'knowledgeBaseId': kbId,
                'modelArn': model_arn,
                'orchestrationConfiguration': {
                    'queryTransformationConfiguration': {
                        'type': 'NONE'
                    },
                },
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'numberOfResults': 5,
                        'overrideSearchType': 'HYBRID',
                        'rerankingConfiguration': {
                            'bedrockRerankingConfiguration': {
                                'metadataConfiguration': {
                                    'selectionMode': 'ALL',
                                },
                                'modelConfiguration': {
                                    'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.rerank-v1:0'
                                },
                                'numberOfRerankedResults': 1
                            },
                            'type': 'BEDROCK_RERANKING_MODEL'
                        }
                    }
                }
            },
            'type': 'KNOWLEDGE_BASE'
        },
        'sessionConfiguration': {
            # KMS Key ARN can be optional. Include only if necessary.
            'kmsKeyArn': 'arn:aws:kms:us-west-2:664418992117:key/6e3b3e5f-9fab-40ee-aa0d-2b7bd35a1621'
        }
    }

    # Use session ID if provided
    if session_id:
        params['sessionId'] = session_id

    try:
        # Call the `retrieve_and_generate_stream` API operation
        response = client.retrieve_and_generate_stream(**params)
        print("Streaming Response:")
        
        # Process events from the response stream
        for event in response['stream']:
            # Extract the generated output and citations
            output_text = event.get('output', {}).get('text', None)
            if output_text:
                print(f"Generated Output: {output_text}")

            stream_citation = event.get('output', {}).get('citation', None)
            if stream_citation:
                print(f"Stream Citation: {stream_citation}")

    except Exception as e:
        print("An error occurred during streaming:")
        traceback.print_exc()
        return

    try:
        # Process and display parts of the response (non-streaming output, if available)
        if 'output' in response and 'text' in response['output']:
            generated_text = response['output']['text']
            print("Generated Text:")
            pp.pprint(generated_text)
        else:
            print("No generated text found in the response.")

        if 'guardrailAction' in response:
            guardRailAction = response['guardrailAction']
            print("Guardrail Action:")
            pp.pprint(guardRailAction)

        if 'sessionId' in response:
            sessionId = response['sessionId']
            print(f"New Session ID: {sessionId}")

        if 'citations' in response:
            citations = response["citations"]
            contexts = []
            for citation in citations:
                if 'retrievedReferences' in citation:
                    retrievedReferences = citation["retrievedReferences"]
                    for reference in retrievedReferences:
                        if 'content' in reference and 'text' in reference['content']:
                            contexts.append(reference["content"]["text"])
            if contexts:
                print("Retrieved Contexts:")
                pp.pprint(contexts)
            else:
                print("No contexts found in response.")
        else:
            print("No citations found in the response.")
        
    except Exception as e:
        print(f"Error when processing response: {str(e)}")

# Example usage
if __name__ == "__main__":
    try:
        input_text = "What is food safety?"
        kbId = 'QJRLZNDR9U'
        model_arn = 'arn:aws:bedrock:us-west-2:664418992117:imported-model/urypmrpq5gjs'
        query_bedrock_agent(kbId, input_text, model_arn)
    except Exception as e:
        print(f"Error: {str(e)}")

An error occurred during streaming:


Traceback (most recent call last):
  File "/tmp/ipykernel_3492/140850324.py", line 77, in query_bedrock_agent
    response = client.retrieve_and_generate_stream(**params)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py", line 569, in _api_call
    return self._make_api_call(operation_name, kwargs)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/client.py", line 1023, in _make_api_call
    raise error_class(parsed_response, operation_name)
botocore.errorfactory.ValidationException: An error occurred (ValidationException) when calling the RetrieveAndGenerateStream operation: 1 validation error detected: Value 'NONE' at 'retrieveAndGenerateConfiguration.knowledgeBaseConfiguration.orchestrationConfiguration.queryTransformationConfiguration.type' failed to satisfy constraint: Member must satisfy enum value set: [QUERY_DECOMPOSITION]


In [48]:
import boto3
import pprint
import traceback

pp = pprint.PrettyPrinter(indent=2)

client = boto3.client('bedrock-agent-runtime', region_name='us-west-2')

sessionId = None

def query_bedrock_agent(kbId, input_text, model_arn, session_id=None):
    global sessionId

    if sessionId:
        params["sessionId"] = session_id

    try:
        params = {
            'input': {
                'text': input_text
            },
            # Simplified template using $search_results directly without lstrip or additional placeholders
            'retrieveAndGenerateConfiguration': {
                'knowledgeBaseConfiguration': {
                    'generationConfiguration': {
                        'guardrailConfiguration': {
                            'guardrailId': '6nsnq854bfxa',
                            'guardrailVersion': 'DRAFT'
                        },
                        'promptTemplate': { '$search_results': {} }
                    }
                },
                'reproductionConfiguration': {
                    'sequenceConfiguration': {
                        'reproductionSequenceConfiguration': {
                            'types': [ 'stream' ]
                        }
                    }
                },
                'retrievalConfiguration': {
                    'vecSearchConfiguration': {
                        'vectorSize': 5
                    }
                }
            }
        }

        print("Streaming Response:")
        
        # Process events from the response stream
        for event in response['stream']:
            # Extract the generated output and citations
            output_text = event.get('output', {}).get('text', None)
            if output_text:
                print(f"Generated Output: {output_text}")

            stream_citation = event.get('output', {}).get('citation', None)
            if stream_citation:
                print(f"Stream Citation: {stream_citation}")

    except Exception as e:
        print("An error occurred during streaming:")
        traceback.print_exc()
        return

    try:
        # Process and display parts of the response (non-streaming output, if available)
        if 'output' in response and 'text' in response['output']:
            generated_text = response['output']['text']
            print("Generated Text:")
            pp.pprint(generated_text)
        else:
            print("No generated text found in the response.")

        if 'guardrailAction' in response:
            guardRailAction = response['guardrailAction']
            print("Guardrail Action:")
            pp.pprint(guardRailAction)

        if 'sessionId' in response:
            sessionId = response['sessionId']
            print(f"New Session ID: {sessionId}")

        if 'citations' in response:
            citations = response["citations"]
            contexts = []
            for citation in citations:
                if 'retrievedReferences' in citation:
                    retrievedReferences = citation["retrievedReferences"]
                    for reference in retrievedReferences:
                        if 'content' in reference and 'text' in reference['content']:
                            contexts.append(reference["content"]["text"])
            if contexts:
                print("Retrieved Contexts:")
                pp.pprint(contexts)
            else:
                print("No contexts found in response.")
        else:
            print("No citations found in the response.")
        
    except Exception as e:
        print(f"Error when processing response: {str(e)}")

# Example usage
if __name__ == "__main__":
    try:
        input_text = "What is food safety?"
        kbId = 'QJRLZNDR9U'
        model_arn = 'arn:aws:bedrock:us-west-2:664418992117:imported-model/urypmrpq5gjs'
        query_bedrock_agent(kbId, input_text, model_arn)
    except Exception as e:
        print(f"Error: {str(e)}")

Streaming Response:
An error occurred during streaming:


Traceback (most recent call last):
  File "/tmp/ipykernel_3492/2581493931.py", line 51, in query_bedrock_agent
    for event in response['stream']:
NameError: name 'response' is not defined


# WIP

In [58]:
import boto3
import pprint
import traceback

pp = pprint.PrettyPrinter(indent=2)

client = boto3.client('bedrock-agent-runtime', region_name='us-west-2')

sessionId = None

def query_bedrock_agent(kbId, input_text, model_arn, session_id=None):
    global sessionId

    params = {
        'input': {
            'text': input_text
        },
        'retrieveAndGenerateConfiguration': {
            'knowledgeBaseConfiguration': {
                'generationConfiguration': {
                    'guardrailConfiguration': {
                        'guardrailId': '6nsnq854bfxa',
                        'guardrailVersion': 'DRAFT'
                    },
                
                'promptTemplate': {
                    'textPromptTemplate': "$search_results$ $query$"
                 }
            
                },
                'knowledgeBaseId': kbId,
                'modelArn': model_arn,
                 'orchestrationConfiguration': {
#                      'queryTransformationConfiguration': {
#                          'type': 'QUERY_DECOMPOSITION'
#                       },
                'promptTemplate': {
#                     'textPromptTemplate': 'You are a query creation agent. You will be provided with a function and a description of what it searches over. The user will provide you a question, and your job is to determine the optimal query to use based on the user question.'
                'textPromptTemplate':"$conversation_history$ $output_format_instructions$ $query$" 
                }
                  },
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'numberOfResults': 5,
                        'overrideSearchType': 'HYBRID',
                        'rerankingConfiguration': {
                        'bedrockRerankingConfiguration': {
                            'metadataConfiguration': {
                                'selectionMode': 'ALL',
                            },
                            'modelConfiguration': {
                                'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.rerank-v1:0'
                            },
                            'numberOfRerankedResults': 1
                        },
                        'type': 'BEDROCK_RERANKING_MODEL'
                      }
                    }
                }
            },
            'type': 'KNOWLEDGE_BASE'
        },
        'sessionConfiguration':{
        'kmsKeyArn': 'arn:aws:kms:us-west-2:664418992117:key/6e3b3e5f-9fab-40ee-aa0d-2b7bd35a1621'
        }
    }

    if session_id:
        params['sessionId'] = session_id

    try:
        response = client.retrieve_and_generate_stream(**params)
#        print(f"SessionId: {response['sessionId']}")
        for event in response['stream']:
            # Each event is a dictionary containing the event data
            output_text = event.get('output', 'text')
            print(f"Output_text: {output_text}")
            print(f"stream_citation: {stream_citation}")

    except Exception as e:
        traceback.print_exc()
        return

    # Process and display the response
    try:
        if 'output' in response and 'text' in response['output']:
            generated_text = response['output']['text']
            pp.pprint(generated_text)
        else:
            print("No generated text found in response")

        if 'guardrailAction' in response:
            guardRailAction = response['guardrailAction']
            pp.pprint(guardRailAction)
        else:
            print("No guardrail action found in response")

        if 'sessionId' in response:
            sessionId = response['sessionId']
            pp.pprint(sessionId)
        else:
            print("No session ID found in response")

        if 'citations' in response:
            citations = response["citations"]
            contexts = []
            for citation in citations:
                if 'retrievedReferences' in citation:
                    retrievedReferences = citation["retrievedReferences"]
                    for reference in retrievedReferences:
                        if 'content' in reference and 'text' in reference['content']:
                            contexts.append(reference["content"]["text"])
            if contexts:
                pp.pprint(contexts)
            else:
                print("No contexts found in response")
        else:
            print("No citations found in response")
    except Exception as e:
        print(f"Error: {str(e)}")

# Example usage
if __name__ == "__main__":
    input_text = "what is food safety?"
    kbId='QJRLZNDR9U'
#     model_arn = 'arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-5-sonnet-20241022-v2:0'
    model_arn = 'arn:aws:bedrock:us-west-2:664418992117:imported-model/urypmrpq5gjs'
    query_bedrock_agent(kbId, input_text, model_arn)


# In[ ]:






Traceback (most recent call last):
  File "/tmp/ipykernel_3492/3262650667.py", line 74, in query_bedrock_agent
    for event in response['stream']:
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/eventstream.py", line 592, in __iter__
    parsed_event = self._parse_event(event)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/eventstream.py", line 608, in _parse_event
    raise EventStreamError(parsed_response, self._operation_name)
botocore.exceptions.EventStreamError: An error occurred (validationException) when calling the RetrieveAndGenerateStream operation: This action doesn't support the model that you provided. Try again with a supported text or chat model. (Service: BedrockRuntime, Status Code: 400, Request ID: bb47c66e-0037-4598-a85a-e2c1ec73d9f2)


In [59]:
import boto3
import pprint
import traceback

pp = pprint.PrettyPrinter(indent=2)

client = boto3.client('bedrock-agent-runtime', region_name='us-west-2')

sessionId = None

def query_bedrock_agent(kbId, input_text, model_arn, session_id=None):
    global sessionId

    params = {
        'input': {
            'text': input_text
        },
        'retrieveAndGenerateConfiguration': {
            'knowledgeBaseConfiguration': {
                'generationConfiguration': {
                    'guardrailConfiguration': {
                        'guardrailId': '6nsnq854bfxa',
                        'guardrailVersion': 'DRAFT'
                    },
                
                'promptTemplate': {
                    'textPromptTemplate': "$search_results$ $query$"
                 }
            
                },
                'knowledgeBaseId': kbId,
                'modelArn': model_arn,
                 'orchestrationConfiguration': {
#                      'queryTransformationConfiguration': {
#                          'type': 'QUERY_DECOMPOSITION'
#                       },
'promptTemplate': {
    'textPromptTemplate': '$conversation_history$ You are a query creation agent. You will be provided with a function ($function_name$) and a description of what it searches over ($description$). The user will provide you a question ($query$), and your job is to determine the optimal query to use based on the user question. Please respond with the optimal query in the format: $output_format_instructions$'
}
                  },
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'numberOfResults': 5,
                        'overrideSearchType': 'HYBRID',
                        'rerankingConfiguration': {
                        'bedrockRerankingConfiguration': {
                            'metadataConfiguration': {
                                'selectionMode': 'ALL',
                            },
                            'modelConfiguration': {
                                'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.rerank-v1:0'
                            },
                            'numberOfRerankedResults': 1
                        },
                        'type': 'BEDROCK_RERANKING_MODEL'
                      }
                    }
                }
            },
            'type': 'KNOWLEDGE_BASE'
        },
        'sessionConfiguration':{
        'kmsKeyArn': 'arn:aws:kms:us-west-2:664418992117:key/6e3b3e5f-9fab-40ee-aa0d-2b7bd35a1621'
        }
    }

    if session_id:
        params['sessionId'] = session_id

    try:
        response = client.retrieve_and_generate_stream(**params)
#        print(f"SessionId: {response['sessionId']}")
        for event in response['stream']:
            # Each event is a dictionary containing the event data
            output_text = event.get('output', 'text')
            print(f"Output_text: {output_text}")
            print(f"stream_citation: {stream_citation}")

    except Exception as e:
        traceback.print_exc()
        return

    # Process and display the response
    try:
        if 'output' in response and 'text' in response['output']:
            generated_text = response['output']['text']
            pp.pprint(generated_text)
        else:
            print("No generated text found in response")

        if 'guardrailAction' in response:
            guardRailAction = response['guardrailAction']
            pp.pprint(guardRailAction)
        else:
            print("No guardrail action found in response")

        if 'sessionId' in response:
            sessionId = response['sessionId']
            pp.pprint(sessionId)
        else:
            print("No session ID found in response")

        if 'citations' in response:
            citations = response["citations"]
            contexts = []
            for citation in citations:
                if 'retrievedReferences' in citation:
                    retrievedReferences = citation["retrievedReferences"]
                    for reference in retrievedReferences:
                        if 'content' in reference and 'text' in reference['content']:
                            contexts.append(reference["content"]["text"])
            if contexts:
                pp.pprint(contexts)
            else:
                print("No contexts found in response")
        else:
            print("No citations found in response")
    except Exception as e:
        print(f"Error: {str(e)}")

# Example usage
if __name__ == "__main__":
    input_text = "what is food safety?"
    kbId='QJRLZNDR9U'
#     model_arn = 'arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-5-sonnet-20241022-v2:0'
    model_arn = 'arn:aws:bedrock:us-west-2:664418992117:imported-model/urypmrpq5gjs'
    query_bedrock_agent(kbId, input_text, model_arn)

Traceback (most recent call last):
  File "/tmp/ipykernel_3492/1249942403.py", line 73, in query_bedrock_agent
    for event in response['stream']:
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/eventstream.py", line 592, in __iter__
    parsed_event = self._parse_event(event)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/eventstream.py", line 608, in _parse_event
    raise EventStreamError(parsed_response, self._operation_name)
botocore.exceptions.EventStreamError: An error occurred (validationException) when calling the RetrieveAndGenerateStream operation: This action doesn't support the model that you provided. Try again with a supported text or chat model. (Service: BedrockRuntime, Status Code: 400, Request ID: 63326716-03b8-4fb5-a1b8-f386d00103ee)
